Future is a container that can hold either the result of a computation or an error that occurred during that computation. When a future is created, it starts in a `PENDING` state. The library does not intend for this object to be created manually, except perhaps for testing purposes.

In [13]:
import concurrent.futures as futures

f = futures.Future()
assert(f._result is None)
assert(f._exception is None)
assert(f._state == 'PENDING')

The `PENDING` status indicates that a computation requested by the user has been registered in the thread pool and placed in a queue, but it has not yet been picked up by any thread for execution. Once a free thread takes the task (callback) from the queue, the future transitions to the `RUNNING` state. A future can only be canceled while it is in the `PENDING` state. Therefore, there is a window of time between the `PENDING` and `RUNNING` states during which the requested computation can be canceled.

In [14]:
import concurrent.futures as futures

def should_cancel_pending_future():
    f = futures.Future()
    assert(f._state == 'PENDING')
    assert(f.cancel())
    assert(f._state == 'CANCELLED')

def should_not_cancel_running_future():
    f = futures.Future()
    f.set_running_or_notify_cancel()
    assert(f._state == 'RUNNING')
    assert(not f.cancel())

def cancel_is_idempotent():
    f = futures.Future()
    assert(f.cancel())
    assert(f.cancel())


should_cancel_pending_future()
should_not_cancel_running_future()
cancel_is_idempotent()

A requested operation in the thread pool can either complete with a computed value or result in an error. Regardless of the outcome, the future transitions to the `FINISHED` state. The result or error is then stored in the corresponding fields.

In [15]:
import concurrent.futures as futures

def future_completed_with_result():
    f = futures.Future()
    f.set_result('foo')
    assert(f._state == 'FINISHED')
    assert(f._result == 'foo')
    assert(f._exception is None)

def future_completed_with_exception():
    f = futures.Future()
    f.set_exception(NameError())
    assert(f._state == 'FINISHED')
    assert(f._result is None)
    assert(isinstance(f._exception, NameError))

future_completed_with_result()
future_completed_with_exception()


To retrieve the result of a computation, the `result` method is used. If the computation is not yet complete, this method will block the current thread (from which `result` was called) until the computation finishes or the wait times out.

If the computation completes successfully without errors, the `result` method returns the computed value.

In [16]:
import concurrent.futures as futures
import time
import threading

f = futures.Future()
def target():
    time.sleep(1)
    f.set_result('foo')
threading.Thread(target=target).start()
assert(f.result() == 'foo')

If an exception occurred during the computation, `result` will raise that exception.

In [17]:
import concurrent.futures as futures
import time
import threading

f = futures.Future()
def target():
    time.sleep(1)
    f.set_exception(NameError)
threading.Thread(target=target).start()
try:
    f.result()
    assert(False)
except NameError:
    assert(True)

If the method times out while waiting, a `TimeoutError` is raised.

In [18]:
import concurrent.futures as futures

f = futures.Future()
try:
    f.result(1)
    assert(False)
except TimeoutError:
    assert(f._result is None)
    assert(f._exception is None)
